In [1]:
import requests
import pandas as pd
import json
import pymongo
import os
from dotenv import load_dotenv, dotenv_values

load_dotenv("app.env")

REWARRDS_DB = os.getenv("REWARDS_CONNECTION_URI")
STAGING_DB = os.getenv("STAGING_CONNECTION_URI")
DEV_DB = os.getenv("DEV_CONNECTION_URI")
HOTELS_PROD_DB = os.getenv("PROD_CONNECTION_URI")
VEHO_PROD_DB = os.getenv("VEHO_PROD_CONNECTION_URI")
TRIPLAZE_JWT = os.getenv("TRIPLAZE_JWT")

def get_hotels_from_triplaze():
    url = "https://pyt-extapi-live.tiniva.com/api/ext-property/dashboardProperty?supplierId=1134&status=ALL&pageNo=1&rows=500"
    payload = {}
    headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'Origin': 'https://pyt-ext-live.tiniva.com',
    'Pragma': 'no-cache',
    'Referer': 'https://pyt-ext-live.tiniva.com/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'Sec-GPC': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    'jwt': TRIPLAZE_JWT,
    'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Brave";v="126"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"'
    }
    response = requests.request("GET", url, headers=headers, data=payload)
    return response.text

def generate_tripalze_hotel_report():
    response = get_hotels_from_triplaze()
    json_data = json.loads(response)
    df = pd.DataFrame(json_data['hotelList'])
    df = df.sort_values(by=['city'])
    df.to_csv('./resources/reports/triplaze_hotels.csv', index=False)

def get_hotels_client():
    uri = HOTELS_PROD_DB
    return pymongo.MongoClient(uri)

hotels_client = get_hotels_client()
hotels = hotels_client.get_database('hotels').get_collection('hotel').find(
    {
        'hotelCodeBySourceProvider.TRIPLAZE' : {'$exists' : True}
    }
)

hotel_by_triplaze_hotel_code = {}
for hotel in hotels:
    hotel_by_triplaze_hotel_code[hotel['hotelCodeBySourceProvider']['TRIPLAZE']] = hotel

triplaze_hotels = json.loads(get_hotels_from_triplaze())['hotelList']
triplaze_hotels_by_hotel_code = {}
for triplaze_hotel in triplaze_hotels:
    triplaze_hotels_by_hotel_code[triplaze_hotel['productId']] = triplaze_hotel

for triplaze_hotel in triplaze_hotels:
    hotel_code = triplaze_hotel['productId']
    try:
        hotel = hotel_by_triplaze_hotel_code[str(hotel_code)]
    except KeyError:
        hotel = None
    if(hotel is not None):
        triplaze_hotel['onboarded'] = True
        triplaze_hotel['statusInProduct'] ='ACTIVE' if hotel['status'] == 'y' else 'INACTIVE'
    else:
        triplaze_hotel['onboarded'] = False
        triplaze_hotel['statusInProduct'] ='INACTIVE'

df = pd.DataFrame(triplaze_hotels)
df = df.sort_values(by=['city'])
df.to_csv('./resources/reports/onboard_status_report.csv', index=False)